In [2]:
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import pyodbc
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions


In [3]:
##MAIPU
file_path = Path(r"C:\Users\adm\Desktop\10 COMUNAS\BD_MASTER_10COMUNAS.csv")
bd_master = pd.read_csv(file_path,sep = ';')

In [4]:
#REDUCIR DATAFRAME A SOLO UN RESTAURANT
bd_master.drop_duplicates('URL', keep="last", inplace=True)
bd_master.sort_index(inplace=True) 

In [ ]:
instanteInicial = datetime.now()
#############################################
tlf_no = "Telefono no encontrado"
lista_platos_2 = []
lista_restaurant_2=[]
lista_direccion_2=[]
lista_telefono =[]
contador_restaurante = 1
driver = webdriver.Chrome(executable_path = "./chromedriver.exe")


for indice, fila in bd_master.iterrows():
    driver.get("https://www.google.com")
    time.sleep(2)
    ##OBTENCION DE RESTAURANTES PARA ITERACION
    restaurant_2 = fila['RESTAURANT']
    ubicacion_1= fila["DIRECCION"]
    ubicacion_2 = ubicacion_1.replace("\n","")
    
    search_bar = driver.find_element(By.XPATH, '//input[@class="gLFyf"]')
    search_bar.send_keys(restaurant_2 ," ", ubicacion_2 , " Numero de telefono" )
    search_bar.send_keys(Keys.ENTER)
    time.sleep(2)

    try:
        l = driver.find_element(By.XPATH, '//span[@class="mw31Ze"]')
        s = l.text
        lista_telefono.append(s)
        lista_restaurant_2.append(restaurant_2)
        lista_direccion_2.append(ubicacion_2)
        lista_platos = [lista_restaurant_2 , lista_direccion_2 ,  lista_telefono]
        print(contador_restaurante ,"SE AGREGO EL NUMERO CORRECTAMENTE DEL RESTAURANT: ",restaurant_2)

    except NoSuchElementException :
        lista_telefono.append(tlf_no)
        lista_restaurant_2.append(restaurant_2)
        lista_direccion_2.append(ubicacion_2)
        lista_platos_2= [lista_restaurant_2 , lista_direccion_2 ,  lista_telefono]
        print(contador_restaurante, "NO SE ENCONTRO EL NUMERO DEL RESTAURANT: ", restaurant_2)
    
    contador_restaurante = contador_restaurante+1
    instanteFinal = datetime.now()
    tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
    segundos = tiempo.seconds
    print(tiempo,segundos," Tiempo Consulta")
    time.sleep(1)
    driver.forward()
    
driver.close()

                               
#############################################
instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds
print(tiempo,segundos," Tiempo Consulta")

In [ ]:
##CREACION DATAFRAME DE RESTAURANT X TELEFONO
df_telefonos= pd.DataFrame(lista_platos)
df_telefonos = df_telefonos.transpose()
df_telefonos.columns = ['RESTAURANT','DIRECCION','TELEFONO']
df_telefonos.to_csv("RESTAURANT_TELEFONO.csv", index=False , sep = ';')